<a href="https://colab.research.google.com/github/CJMartinezV/ABI_Rep/blob/main/Action_Wizard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Instalar Whisper de OpenAI y Google AI Studio
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q -U google-generativeai
!pip install -U -q google-generativeai

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [19]:
#Importar paquetes
import google.generativeai as genai
import whisper
from datetime import datetime, timedelta
from google.colab import userdata


# Llama automáticamente a la llave que guardaste en el menú de "Secrets" (icono de llave)
try:
    GOOGLE_API_KEY = userdata.get('APIGemini')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✅ API Key configurada correctamente desde Secrets.")
except:
    print("❌ Error: No se encontró la llave 'GEMINI_KEY' en el menú de Secrets de Colab.")


# Definimos el Prompt Maestro basado en tus instrucciones
PROMPT_LOGISTICA = """
Te vas a comportar como mis asistente profesional,
con conocimientos profundos en logística, específicamente del sector de consumo masivo de cerveza,
una empresa como ABInBev en Colombia (Bavaria). Quiero que cada que yo te especifique "Resumen rutina
tu me entregues un breve resumen y una lista con las acciones (Compromisos que deben ser al menos 3)
con fechas estimadas de cumplimiento (si no se especifica fecha, se deja por defecto 4 días después)
que se deben generar para cada área responsable (PILAR) las cuales deben estar redactadas iniciando
con un verbo y que responda las preguntas qué, quién cómo y para qué. Ten en cuenta información como:
TRI: KPI relacionado con accidentalidad . Total Recordable Injuries
FAT: Fatalities
LTI: Loss Time Injuries
MDI: Modify Duty Injuries
MTI: Medical Teatment Injuries
FAI: First Aid Injuries
SIF: Tema relacionado con riesgo de fatalidad o incidente severo (O prevención del mismo).
Significa Serious Injuries or Fatalities SIF se clasifica en Actual, Potencial y Precursor,
teniendo en cuenta si la ocurrencia de un evento o incidente afectó gravemente (actual),
si ocurrió afectando levemente a las personas (Potencial), o si hubo posibilidad de ocurrencia
pero no hubo afectaciones (Precursor).
Refusal: Rechazo Producto por parte de los clientes, se representa como el % de Hectolitros (HL) rechazados del total despachado.
On Time: Entrega dentro de la promesa que se sostiene con el cliente, su valor es representado por % de cumplimiento.
In Full, pedidos entregados completos, representado con % de cumplimiento.
OTIF: % de pedidos que se entregan completos y a tiempo
OWD: Operational Work Diagnosis, % de adherencia a los procesos
ACI: Actos y Condiciones Inseguras
GOP: Buenas pácticas Operacionales
OPL: One Point Lesson, es un instructivo o folleto conciso y visual sobre un proceso o parte de el.
SOP: Standard Operating Practices, se refiere a Instructivos y manuales detallados para estandarizar procesos de forma completa.
TOR: Terms Of References, Estructura de ejecución de una rutina
Rutina: Actividades o foros que deben ejecutarse con cierta periodicidad por los pilares
Pilar: área encargada de un proceso que apuntan a los resultados y sueños de la compañía,
ABInbev cuenta con 7 Pilares para el DPO, los cuales son
Gente (People), Seguridad (Safety), Almacén (Warehouse), Flota(Fleet), Reparto (Delivery), Planeación (Planning) y Gestión (Management).
VLC: Variable Logistic Costs, Costos por volumen.
HL: Hectolitros
NPS: Net Promoter Score, es un indicador de Satisfacción del cliente. Clasificados como Promotores, Neutrales y Detractores
Matinal: Rutina ejecutada en la primera jornada, Antes de 6 am para socializar información relevante y temas de seguridad al personal operativo.
Diaria: Rutina Diaria Logística, reunión de líderes para evaluar aspectos generales, información relevante y estatus operacional.

Si detectas que te estas ejecutando en una reunion de algún otro corporativo, adaptate y genera las acciones respectivas.

Cada vez que recibas una transcripción, si detectas que es una "Resumen rutina", entrega:
1. Un breve resumen ejecutivo.
2. Una lista de compromisos (mínimo 3).
3. Formato: Acción (Verbo + Qué + Quién + Cómo + Para qué) | Pilar Responsable | Fecha Estimada.
Si no hay fecha, usa por defecto: {fecha_defecto}.
4. Ten en cuenta que Los responsables principales son los pilares de Flota, Almacén, Reparto, Seguridad y Gente.

"""

def procesar_con_ia(texto_transcrito):
    # 1. Definimos la fecha de compromiso por defecto (4 días después)
    fecha_4_dias = (datetime.now() + timedelta(days=4)).strftime('%Y-%m-%d')

    try:
        # CAMBIO CLAVE: Usamos el nombre completo del modelo
        model = genai.GenerativeModel(model_name='models/gemini-flash-latest')

        # 3. Unimos tu Prompt de experto con el texto transcribo
        prompt_final = f"{PROMPT_LOGISTICA.format(fecha_defecto=fecha_4_dias)}\n\nTexto a procesar:\n{texto_transcrito}"

        # 4. Generación del resumen
        response = model.generate_content(prompt_final)
        return response.text

    except Exception as e:
        # Si falla el flash, intentamos con el pro automáticamente
        try:
            model_pro = genai.GenerativeModel(model_name='models/gemini-pro')
            response = model_pro.generate_content(prompt_final)
            return response.text
        except:
            return f"Error persistente en la API: {str(e)}"

✅ API Key configurada correctamente desde Secrets.


In [13]:
# 1. Transcribir con Whisper
model_whisper = whisper.load_model("medium")
print("Transcribiendo audio...")
result = model_whisper.transcribe("/content/Rutina.mp3", language="Spanish")
texto_puro = result["text"]


Transcribiendo audio...


In [20]:
# 2. Generar el resumen estructurado con el perfil de experto
print("Generando resumen logístico...")
resumen_final = procesar_con_ia(texto_puro)

print("\n--- RESULTADO DEL ASISTENTE LOGÍSTICO ---\n")
print(resumen_final)

Generando resumen logístico...

--- RESULTADO DEL ASISTENTE LOGÍSTICO ---

¡Entendido! Me ajusto completamente al rol de su Asistente Profesional con enfoque en la logística de consumo masivo, adaptando la nomenclatura de Pilares de ABInBev (Bavaria) a la estructura y necesidades de la empresa en la que nos encontramos: Pesquera Diamante.

A pesar de que el texto no es una transcripción de una "Rutina" logística diaria (como una Matinal o una Diaria), sino de una reunión estratégica de gerencia, procesaré la información bajo el formato de "Resumen rutina" para establecer los compromisos derivados de las propuestas presentadas.

***

## Resumen rutina

**Resumen Ejecutivo:**

La reunión de gerencia, liderada por Edwin Contreras, se centró en la estrategia de crecimiento y las innovaciones necesarias para afrontar los cambios post-pandemia. Los líderes de Producción, Ventas, Marketing y Cadena de Suministro presentaron propuestas clave enfocadas en la expansión de capacidades operacional